In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from cartiflette import carti_download
import plotly.express as px
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

## Import Données

In [ ]:
df_patientele = pd.read_csv("Bases de données/patientele.csv", sep=";")
df_patientele['nombre_patients_uniques'] = pd.to_numeric(df_patientele['nombre_patients_uniques'],errors='coerce')
df_patientele.columns = df_patientele.columns.str.replace("\ufeff", "", regex=False).str.strip()

df_patientele_2017 = df_patientele[(df_patientele['annee'] == 2017) & (df_patientele['profession_sante'] == "Médecins généralistes (hors médecins à expertise particulière - MEP)")]
df_patientele_2017 = df_patientele_2017[["departement","libelle_departement","patients_uniques_integer"]]
df_patientele_2017.columns = ["Code Département","Département","Nb de patients unique par spécialiste"]
df_patientele_2017.head() 

In [ ]:
df_Indicateurs_département = pd.read_excel("Bases de données/data (1).xlsx")
#df_Indicateurs_département = df_Indicateurs_département[["dep", "codgeo", "libgeo", "p21_pop"]]
#df_commune.reset_index(drop=True, inplace=True)
df_Indicateurs_département.columns = ["Code Département", "Libellé Département", "Nombre de décès domiciliés 2024", "Salaire net EQTP mensuel moyen 2023","Espérance de vie des hommes à 60 ans 2024",	"Espérance de vie des femmes à 60 ans","Taux d'activité","Taux de chômage annuel moyen 2024","Part des résidences principales en suroccupation","Taux de pauvreté - familles monoparentales 2021","Taux de pauvreté 2021","Médiane du niveau de vie 2021","Taux brut de mortalité 2024","Taux de natalité 2024"]
df_Indicateurs_département = df_Indicateurs_département.sort_values(by="Code Département")

df_Indicateurs_département.head()

## Merging & Nettoyage

In [ ]:
df_Indicateurs_département["Code Département"] = (df_Indicateurs_département["Code Département"].astype(str).str.zfill(2))
df_patientele_2017["Code Département"] = (df_patientele_2017["Code Département"].astype(str).str.zfill(2))

In [ ]:
df = df_patientele_2017.merge(df_Indicateurs_département, on="Code Département", how="inner")
df = df.drop(columns=["Libellé Département"])
df = df.replace("N/A - résultat non disponible",np.nan)
df.head()

In [ ]:
cols_num = ["Nb de patients unique par spécialiste", "Nombre de décès domiciliés 2024", "Salaire net EQTP mensuel moyen 2023","Espérance de vie des hommes à 60 ans 2024",	"Espérance de vie des femmes à 60 ans","Taux d'activité","Taux de chômage annuel moyen 2024","Part des résidences principales en suroccupation","Taux de pauvreté - familles monoparentales 2021","Taux de pauvreté 2021","Médiane du niveau de vie 2021","Taux brut de mortalité 2024","Taux de natalité 2024"]

for col in cols_num:
    df[col] = pd.to_numeric(df[col],errors="coerce")

print(df.shape)
df.head()

## Régression linéaire

In [ ]:
y = df["Nb de patients unique par spécialiste"]

In [ ]:
X_cols = [
    "Salaire net EQTP mensuel moyen 2023",
    "Taux de chômage annuel moyen 2024",
    "Médiane du niveau de vie 2021",
    "Taux de natalité 2024",
    "Espérance de vie des hommes à 60 ans 2024",
    "Part des résidences principales en suroccupation"
]

In [ ]:
X = df[X_cols]
df_reg = pd.concat([y, X], axis=1).dropna()
print(df_reg.shape)

y_reg = df_reg[y.name]
X_reg = df_reg[X_cols]

In [ ]:
X_reg = sm.add_constant(X_reg)

In [ ]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(df_reg[X_cols])

X_scaled = pd.DataFrame(
    X_scaled,
    columns=X_cols,
    index=df_reg.index
)

X_scaled = sm.add_constant(X_scaled)

model = sm.OLS(y_reg, X_scaled)
results = model.fit()

print(results.summary())

In [ ]:
coef = results.params
conf = results.conf_int()
conf.columns = ["lower", "upper"]

coef = coef.drop("const")
conf = conf.drop("const")

plt.figure(figsize=(8, 5))
plt.errorbar(
    coef,
    coef.index,
    xerr=[coef - conf["lower"], conf["upper"] - coef],
    fmt="o"
)
plt.axvline(0, color="black", linestyle="--")
plt.title("Effets estimés sur le nombre de patients par médecin")
plt.xlabel("Coefficient (OLS)")
plt.ylabel("")
plt.tight_layout()
plt.show()


In [ ]:
y_pred = results.fittedvalues

plt.figure(figsize=(6, 6))
plt.scatter(y_reg, y_pred, alpha=0.7)
plt.plot(
    [y_reg.min(), y_reg.max()],
    [y_reg.min(), y_reg.max()],
    linestyle="--"
)

plt.xlabel("Valeurs observées")
plt.ylabel("Valeurs prédites")
plt.title("Qualité d'ajustement du modèle")
plt.tight_layout()
plt.show()
